**Nowell Kevin Reyes - 20658133**

# Task 1. Regression 

**Motivation**: Using the dataset `bloodpressure-23.csv`, we will compare different regression models and choose the "best model fit" that **TODO**


Our first task is to load the data and view its properties. We will use the `numpy` and `pandas` libraries to do this. 

*Note to the marker*: I have also included the `os` module to ensure that the workbook will load in the same folder as the data file (in case of issues).

In [4]:
# Import required libraries
import numpy as np
import pandas as pd
import os

os.chdir('.') # set working directory to current folder

In [5]:
# load the data
df = pd.read_csv("bloodpressure-23.csv")

# view the head of the data
df.head()

,ID-NUMBER,AGE,ED-LEVEL,SMOKING STATUS,EXERCISE,WEIGHT,SERUM-CHOL,SYSTOLIC,IQ,SODIUM,GENDER,MARITAL-STATUS,NAME
0,1,27,2,1,1,120,193,126,118,136,F,M,"Braund, Mr. Owen Harris"
1,2,18,1,0,1,145,210,120,105,137,M,S,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,3,32,2,0,0,118,196,128,115,135,F,M,"Heikkinen, Miss. Laina"
3,4,24,2,0,1,162,208,129,108,142,M,M,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,5,19,1,2,0,106,188,119,106,133,F,S,"Allen, Mr. William Henry"
